# **Launch Sites Locations Analysis with Folium**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using matplotlib and seaborn and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using Folium.

## Objectives

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:

In [60]:
import folium
import pandas as pd

In [61]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

In [62]:
# Import dataset

spacex_df=pd.read_csv('spacex_Folium.csv')

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.

In [63]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [64]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [65]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon= DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map

In [66]:
# Initial the map

#site_map = folium.Map(location=sites, zoom_start=5)

site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


fg = folium.FeatureGroup()

list  = launch_sites_df[['Launch Site','Lat', 'Long']].values

# loop through the 100 crimes and add each to the incidents feature group
for i in list:
     fg.add_child(
         folium.Circle( location = [i[1], i[2]],radius= 50, color='#d35400', fill=True).add_child(folium.Popup(i[0])),)

for i in list:
      fg.add_child(
      folium.Marker(
            location = [i[1], i[2]],popup = i[0],icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % i[0], ),)
      )
    

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

site_map.add_child(fg)

Now, you can explore the map by zoom-in/out the marked areaNow, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


* The launch sites are all in proximity to the equator line beacuse they take optimum advantage of the Earth's substantial rotational speed for the spaceship launching.
* All the launch sites are close to coastal lines because with an ocean downrange, any launch failures will splash down there, with minimal risk to life and property compared to if it crashes on land

## Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [67]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [68]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value

In [69]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# create column out_come to label sucessesful or failed lauching

spacex_df
marker_color = []
out_come = []

for x in spacex_df['class']:
    if x == 1 :
        marker_color.append('green')
        out_come.append('Succes')
       
    else :
      marker_color.append('red')
      out_come.append('Fail')
        
        
marker_color

spacex_df['marker_color'] = marker_color
spacex_df['out_come'] = out_come
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color,out_come
0,CCAFS LC-40,28.562302,-80.577356,0,red,Fail
1,CCAFS LC-40,28.562302,-80.577356,0,red,Fail
2,CCAFS LC-40,28.562302,-80.577356,0,red,Fail
3,CCAFS LC-40,28.562302,-80.577356,0,red,Fail
4,CCAFS LC-40,28.562302,-80.577356,0,red,Fail


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [70]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)



# for each row in spacex_df data frame

# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    
    folium.Marker(location = [record[1],record[2]],icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % record[0], ))
                  
                 
    marker = folium.Marker( location= [record[1],record[2]],
                     popup = record[5],
                       icon=folium.Icon(color=record[4]))
                           
                        
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates, you can also click on each marker to see the outcome.

## TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [71]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [72]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [73]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon
print('VAFB SLC-4E =',calculate_distance(34.632834, -120.610745, 34.63558, -120.62593))
print('CCAFS SC-40 =',calculate_distance(28.562302,-80.577356, 28.56222, -80.56781))
print('CCAFS SLC-40 =',calculate_distance(28.563197, -80.576820, 28.56354, -80.56796))


VAFB SLC-4E = 1.4228940873579703
CCAFS SC-40 = 0.9326214967547038
CCAFS SLC-40 = 0.8663908470112106


In [74]:
#Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 

distance_marker = folium.Marker(
                   location = [28.56354, -80.56796],
                   icon=DivIcon(
                   icon_size=(20,20),
                   icon_anchor=(0,0),
                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(0.8663908470112104),
                           )
                         )

site_map.add_child(distance_marker)

In [75]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=([28.563197, -80.576820],[28.56354, -80.56796]),weight=1)
site_map.add_child(lines)

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first

In [76]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

print ('Distance to the closest railway = ',calculate_distance(34.632834, -120.610745, 34.63411, -120.62473))
print('Distance to the closest city = ',calculate_distance(28.573255,-80.646895, 28.57606,-80.80101))
print('Distance to the closest highway = ',calculate_distance(28.563197, -80.576820,  28.56345,-80.57081))
print('Distance to the closest coastline =',calculate_distance(28.563197, -80.576820, 28.56354, -80.56796))

Distance to the closest railway =  1.2877580980596526
Distance to the closest city =  15.05740702348411
Distance to the closest highway =  0.5878027167468074
Distance to the closest coastline = 0.8663908470112106


In [77]:
# Map distance to the closes Railway 


distance_marker = folium.Marker(
                   location = [34.63411, -120.62473],
                   icon=DivIcon(
                   icon_size=(20,20),
                   icon_anchor=(0,0),
                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(1.2877580980596526),
                           )
                         )
lines=folium.PolyLine(locations=([34.632834, -120.610745],[34.63411, -120.62473]),weight=1)

site_map.add_child(lines)
site_map.add_child(distance_marker)

In [78]:
# Map distance to the closes city

distance_marker = folium.Marker(
                   location = [28.57606,-80.80101],
                   icon=DivIcon(
                   icon_size=(20,20),
                   icon_anchor=(0,0),
                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(15.05740702348411),
                           )
                         )
lines=folium.PolyLine(locations=([28.573255,-80.646895],[28.57606,-80.80101]),weight=1)

site_map.add_child(lines)
site_map.add_child(distance_marker)

In [79]:
# Map distance to the closes High way

distance_marker = folium.Marker(
                   location = [28.56345,-80.57081],
                   icon=DivIcon(
                   icon_size=(20,20),
                   icon_anchor=(0,0),
                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(0.5878027167468074),
                           )
                         )
lines=folium.PolyLine(locations=([28.563197, -80.576820],[28.56345,-80.57081]),weight=1)

site_map.add_child(lines)
site_map.add_child(distance_marker)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.

* They are close to railways because big rockets require lots of big pieces to be brought in for assembly and railway is the cheapest for of transportation. 
* They are close to some highways for easy accesibility for personnels who might be living in cities far from the sites 
* They are close to coastlines because they use ports for transportation of large pieces and also minimizing the risk of having any debris dropping or exploding near people.
* Launch site keep certain distance from people because they minimizing the risk of having any debris dropping or exploding near people .